In [ ]:
model_location = 'C:\\Users\\Public\\Documents\\Data\\2018\\vqa_models\\20180629_0933_17\\vqa_model_NLP.h5'
strategy_str = 'NLP'

In [1]:
# %%capture
import os
import numpy as np
from pandas import HDFStore
from vqa_logger import logger 
from enum import Enum
from keras.models import load_model

c:\local\Anaconda3-4.1.1-Windows-x86_64\envs\conda_env\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
with VerboseTimer("Loading Model"):
    model = load_model(model_location)